In [20]:
# Core imports
import json
import sys
from pprint import pprint
import pandas as pd

sys.path.append("../")
# LLM Engine imports
from Src.LLM.llm_client import OllamaLLMClient
from Src.LLM.llm_engine import LLMEngine
from Src.Pathway_1.pathway1 import pathway_1_lookup as pathway1

llm_client = OllamaLLMClient()
llm_engine = LLMEngine(llm_client)

print("LLM Engine initialized successfully ")



LLM Engine initialized successfully 


In [21]:
import Src.Pathway_1
print("Functions found in Pathway_1.py:")
print([f for f in dir(Src.Pathway_1) if not f.startswith("__")])

Functions found in Pathway_1.py:
['pathway1']


In [ ]:
class NutriSenseRouter:
    def __init__(self, df, llm_engine, image_model=None):
        self.df = df
        self.engine = llm_engine
        self.image_model = image_model # Your trained CNN

    def classify_intent(self, query):
        """Uses Llama to decide the pathway."""
        prompt = f"""
        Analyze this user query: "{query}"
        Categorize into:
        - "COMPARE": Mentioning two or more foods.
        - "MODIFY": Asking for a healthy version/change (e.g. 'less oil', 'vegan').
        - "EXTRACT": Just asking for info on one food.

        Return ONLY a JSON object:
        {{"pathway": "COMPARE"|"MODIFY"|"EXTRACT", "dishes": ["dish1", "dish2"], "constraint": "modifier text or null"}}
        """
        response = self.engine.llm.generate(prompt)
        try:
            # Extract JSON from potential LLM chatter
            data = json.loads(response[response.find("{"):response.rfind("}")+1])
            return data
        except:
            return {"pathway": "EXTRACT", "dishes": [query], "constraint": None}
    
    def execute(self, text_query=None, image_input=None):
         # PATHWAY 4: IMAGE
        if image_input is not None:
            dish_name, img_conf = self.image_model.predict(image_input) 
            return self.handle_extraction(dish_name, override_conf=img_conf)

        # TEXT PROCESSING
        intent = self.classify_intent(text_query)
    
        # Use .get('constraint') instead of ['constraint'] to avoid KeyErrors
        dishes = intent.get('dishes', [])
        constraint = intent.get('constraint') # Defaults to None if missing

        if intent.get('pathway') == "COMPARE" and len(dishes) >= 2:
         return self.handle_comparison(dishes, constraint)
    
        elif intent.get('pathway') == "MODIFY":
         # Make sure there is at least one dish before accessing index 0
            target_dish = dishes[0] if dishes else text_query
            return self.handle_modification(target_dish, constraint)
    
        else:
            target = dishes[0] if dishes else text_query
            return self.handle_extraction(target)  

    # Logic Handlers
    def handle_extraction(self, name, override_conf=None):
        res = pathway1(name, self.df)
        if res['status'] == "FOUND":
            out = res['results'][0]
            out['accuracy'] = (override_conf * 100) if override_conf else (out['confidence'] * 100)
            return out
        return self.engine.estimate_nutrition(name)

    def handle_modification(self, name, constraint):
        res = pathway1(name, self.df)
        if res['status'] == "FOUND":
            d = res['results'][0]
            out = self.engine.modify_recipe(d['recipe_name'], d['nutrition'], d['ingredients'], d['instructions'], constraint)
            out['accuracy'] = d['confidence'] * 100
            return out
        return self.engine.estimate_nutrition(f"{name} with {constraint}")

    def handle_comparison(self, dishes, goal):
        res_a = pathway1(dishes[0], self.df)
        res_b = pathway1(dishes[1], self.df)
        if res_a['status'] == "FOUND" and res_b['status'] == "FOUND":
            out = self.engine.compare_dishes(res_a['results'][0]['recipe_name'], res_a['results'][0]['nutrition'],
                                             res_b['results'][0]['recipe_name'], res_b['results'][0]['nutrition'], goal)
            out['accuracy'] = ((res_a['results'][0]['confidence'] + res_b['results'][0]['confidence'])/2) * 100
            return out
        return self.engine.estimate_nutrition(f"Compare {dishes[0]} and {dishes[1]}")

In [23]:
# 1. Setup Mock Dataset
data = {
    'best_match_clean': ['dosa', 'biryani', 'paneer tikka'],
    'final_food_name': ['Masala Dosa', 'Hyderabadi Biryani', 'Paneer Tikka'],
    'Calories (kcal)': [150, 450, 250],
    'Protein (g)': [4, 15, 12],
    'TranslatedIngredients': "Rice, spices, oil",
    'TranslatedInstructions': "Cook on tawa"
}
df = pd.DataFrame(data)

# 2. Setup Mock Image Model
class MockImageModel:
    def predict(self, img_path):
        return "dosa", 0.98 # Returns predicted dish and confidence

# 3. Initialize Router
client = OllamaLLMClient()
engine = LLMEngine(client)
router = NutriSenseRouter(df, engine, image_model=MockImageModel())

print("NutriSense AI Router Ready.")

NutriSense AI Router Ready.


In [24]:
print("TEST: EXTRACTION ")
res = router.execute(text_query="Tell me about Dosa")
print(f"Dish: {res.get('recipe_name')} | Accuracy: {res.get('accuracy')}%")

TEST: EXTRACTION 
Dish: Masala Dosa | Accuracy: 95.0%


In [25]:
print("--- TEST: MODIFICATION ---")
res = router.execute(text_query="Make Biryani healthier with less oil")
print(f"Pathway: {res.get('pathway')}")
print(f"AI Suggestion: {res.get('llm_response')}")

--- TEST: MODIFICATION ---
Pathway: modification
AI Suggestion: Here's the modified recipe:

**1. Modified Ingredients:**
* Rice
* Spices
* Low-fat oil or ghee (reduced from original amount)

**2. Modified Cooking Method:**
* Cook on tawa, but use a non-stick pan to reduce oil absorption.
* Use a small amount of low-fat oil or ghee for cooking and add it in stages to prevent excessive oil usage.

**3. Changes Summary:**

• Reduced the amount of oil used in cooking by switching to a low-fat option and using it sparingly.
• Adopted a non-stick pan to minimize oil absorption during cooking.
• Kept the core ingredients (rice, spices) intact while modifying only the cooking method and ingredient quantity.

**4. Qualitative Nutrition Impact:**
The modified recipe should result in a slightly healthier version of Hyderabadi Biryani with reduced fat content. However, the exact nutrition numbers will vary depending on the specific low-fat oil or ghee used. The dish is likely to retain its origin

In [26]:
print("--- TEST: COMPARISON ---")
res = router.execute(text_query="Dosa vs Biryani for protein")
print(f"Pathway: {res.get('pathway')}")
print(f"Comparison: {res.get('llm_response')}")

--- TEST: COMPARISON ---
Pathway: comparison
Comparison: **Summary Comparison**

|  | Dish A: Masala Dosa | Dish B: Hyderabadi Biryani |
| --- | --- | --- |
| Calories (kcal) | 150 | 450 |
| Protein (g) | 4 | 15 |

**Better Choice for the Goal: Protein**

Dish B: Hyderabadi Biryani is better suited for a protein goal, with 3.75 times more protein than Dish A.

**Trade-offs**

While choosing Hyderabadi Biryani for a high-protein diet, consider the following trade-offs:

* Higher calorie intake (450 kcal vs 150 kcal in Masala Dosa)
* Increased risk of weight gain due to higher caloric content
* May require additional physical activity or adjustments to overall diet to compensate for the increased calories


In [27]:
print("--- TEST: COMPARISON ---")
res = router.execute(text_query="Dosa vs Biryani")
print(f"Pathway: {res.get('pathway')}")
print(f"Comparison: {res.get('llm_response')}")

--- TEST: COMPARISON ---
Pathway: comparison
Comparison: Here's the comparison of Dish A: Masala Dosa and Dish B: Hyderabadi Biryani based on the user's goal of general health:

**1. Summary Comparison**

| **Nutrient** | **Dish A: Masala Dosa** | **Dish B: Hyderabadi Biryani** |
| --- | --- | --- |
| Calories (kcal) | 150 | 450 |
| Protein (g) | 4 | 15 |
| [Other micronutrients not provided]

**2. Better Choice (for the goal)**

Based on the nutrition values, Dish A: Masala Dosa is a better choice for general health compared to Dish B: Hyderabadi Biryani. Here's why:

* **Calories**: With only 150 calories, Masala Dosa has significantly fewer calories than Hyderabadi Biryani (450 calories), making it a more suitable option for those watching their calorie intake.
* **Protein**: Although both dishes have relatively low protein content, Masala Dosa still provides some essential amino acids.

**3. Trade-offs**

However, there are trade-offs to consider:

* **Higher calorie density**: Hyd

In [28]:
print("--- TEST: FALLBACK ---")
res = router.execute(text_query="Nutrition for Space Dragon Noodles") # Not in dataset
print(f"Source: {res.get('source')}")
print(f"AI Estimate: {res.get('llm_response')}")

--- TEST: FALLBACK ---
Source: llm_estimate
AI Estimate: **Space Dragon Noodles**

1. **Estimated Nutrition (ranges)**:
	* Calories: 400-600 per serving
	* Protein: 20-30g (from noodles, vegetables, and possibly meat or seafood)
	* Fat: 10-15g (from oil, sauces, and potentially added meats or seafood)
	* Carbohydrates: 60-80g (from noodles, vegetables, and sauces)
	* Fiber: 4-6g (from vegetables and whole grain noodles)
2. **Reference Dish Used**: None
3. **Confidence Level**: Low
4. **Disclaimer**: As this dish is not a known or documented culinary creation, the estimated nutrition values are highly speculative and should be taken as rough estimates only.
